<a href="https://colab.research.google.com/github/Henil28P/UAV-BushfireDetection/blob/main/UAV_Bushfire_Detection_CSS2_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
cd /content/drive/Shareddrives/dataset/

[Errno 2] No such file or directory: '/content/drive/Shareddrives/dataset/'
/content


# Data Preprocessing

Creating YOLO model

In [ ]:
!nvidia-smi

Thu Apr 18 11:10:04 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
!pip install ultralytics

#from IPython import display
#display.clear_output()
# !yolo mode=checks

In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

For the next step, we can choose to run either the CLI version or the Python version. Both give the SAME RESULTS

# CLI

model=' ' you can use custom model

source=' ' you can specify video file, URL, device ID

In [ ]:
%cd {home}
!yolo task=detect mode=predict model=yolov8n.pt conf=0.25 source=''

Shows Image

In [ ]:
%cd {HOME}
Image(filename=' ' , height=600)

# Python

In [ ]:
model = YOLO(f'{HOME}/yolov8n.pt')
results = model.predict(source=' ', conf=0.25)

# labeling and training a custom dataset using roboflow

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="QFqjO7WCKoOzTMvDBndP")
project = rf.workspace("css2test").project("css2_fire_smoke_labelling_v2")
version = project.version(3)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.1, to fix: `pip install ultralytics==8.0.196`


In [ ]:
%cd {HOME}

!yolo task=detect mode=train model=yolov8n.pt data=/content/CSS2_fire_smoke_labelling_v2-3/data.yaml epochs=20 imgsz=800

/content
Ultralytics YOLOv8.2.1 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/CSS2_fire_smoke_labelling_v2-3/data.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=800, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_la

names:
- fire
- person
- smoke
- vehicle
nc: 4
roboflow:
  license: CC BY 4.0
  project: css2_fire_smoke_labelling_v2
  url: https://universe.roboflow.com/css2test/css2_fire_smoke_labelling_v2/dataset/3
  version: 3
  workspace: css2test
test: /content/CSS2_fire_smoke_labelling_v2-3/test/images
train: /content/CSS2_fire_smoke_labelling_v2-3/train/images
val: /content/CSS2_fire_smoke_labelling_v2-3/valid/images



In [ ]:
from google.colab import files
files.download('/content/runs/detect/train6')

In [ ]:
from inference_sdk import InferenceHTTPClient

CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="kF987QgdreoXXmF1vc2E"
)

result = CLIENT.infer(your_image.jpg, model_id="fire-smoke-detection-eozii/1")

# Inference with custom model

In [ ]:
%cd {HOME}
!yolo task=detect mode=predict model=/content/runs/detect/train6/weights/best.pt conf=0.25 source=/content/CSS2_fire_smoke_labelling_v2-3/test/images/6-RGB-video_MP4-0000_jpg.rf.69b3d63b1e3aaca98ef8fe3513067948.jpg

/content
Ultralytics YOLOv8.2.1 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3006428 parameters, 0 gradients, 8.1 GFLOPs

image 1/1 /content/CSS2_fire_smoke_labelling_v2-3/test/images/6-RGB-video_MP4-0000_jpg.rf.69b3d63b1e3aaca98ef8fe3513067948.jpg: 480x800 2 fires, 1 smoke, 82.3ms
Speed: 7.5ms preprocess, 82.3ms inference, 2093.3ms postprocess per image at shape (1, 3, 480, 800)
Results saved to runs/detect/predict2
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'{HOME}/runs/detect/predict.3/*.jpg')[: ]:
  display(Image(filename=image_path, width=600))
  print("*\n")

In [ ]:
!yolo task=detect mode=predict model=/content/runs/detect/train6/weights/best.pt conf=0.25 source=/content/4-RBG-video.MP4

# Classification model

In [ ]:
!nvidia-smi

Fri May  3 02:25:12 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
!pip install ultralytics

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="QFqjO7WCKoOzTMvDBndP")
project = rf.workspace("css2test").project("css2_fire_smoke_labelling_v2")
version = project.version(3)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.7, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to CSS2_fire_smoke_labelling_v2-3 in yolov8:: 100%|██████████| 1846/1846 [00:00<00:00, 1858.99it/s]


In [ ]:
%cd {HOME}

!yolo task=classify mode=train model=yolov8n-cls.pt data=/content/CSS2_fire_smoke_labelling_v2-3/data.yaml epochs=20 imgsz=800

/content
Ultralytics YOLOv8.2.7 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=/content/CSS2_fire_smoke_labelling_v2-3, epochs=20, time=None, patience=100, batch=16, imgsz=800, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels

In [ ]:
%cd {HOME}
!yolo task=detect mode=predict model=/content/runs/detect/train6/weights/best.pt conf=0.25 source=/content/CSS2_fire_smoke_labelling_v2-3/test/images/6-RGB-video_MP4-0000_jpg.rf.69b3d63b1e3aaca98ef8fe3513067948.jpg

In [ ]:
!yolo task=detect mode=predict model=/content/runs/detect/train6/weights/best.pt conf=0.25 source=/content/4-RBG-video.MP4

# Experimenting

Modes available:

- Train: Fine-tune model on custom or preloaded dataset
- Val: Post-training checkpoint to validate model performance
- Predict: Apply real-world data on your model
- Track: Extend your object detection model into real-time tracking applications
- Benchmark: Analyse speed and accuracy of model

Training

For more info:
https://docs.ultralytics.com/modes/train/

In [ ]:
# Load model

# Build a new model from YAML
model = YOLO('yolov8n.yaml')

# load a pretrained model (recommended for training)
model = YOLO('yolov8n.pt') # YOLO('path/to/best.pt')

# Build from YAML and transfer weights
model = YOLO('yolov8n.yaml').load('yolov8n.pt')

# Train model
results = model.train(data=' ', epochs=20, imgsz=800)

Validation

For more info: https://docs.ultralytics.com/modes/val/

In [ ]:
# Validate the model
metrics = model.val()
metrics.box.map   # map50-95
metrics.box.map50 # map50
metrics.box.map79 # map75
metrics.box.maps  # a list contains map50-95 of each category

Predicting

For more info: https://docs.ultralytics.com/modes/predict/

In [ ]:
# Define path to video file
source = 'path/to/video.mp4'

# Run inference on the source
results = model(source, stream=True)

# Or

# # Run inference with arguments
model.predict(source=' ', save=True, imgsz=800, conf=0.25)

Export a model (optional)

For more info: https://docs.ultralytics.com/modes/export/

In [ ]:
# Export the model
model.export()